In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import fiona
import os

### Apportion the population with each OA

In [40]:
# File paths
oa_population_shapefile = './data/output_area_population.shp'
oa_age_bands_csv = './data/oa_age_bands.csv'

# Load shapefile and CSV
print("Loading shapefile and CSVs...")
oa_gdf = gpd.read_file(oa_population_shapefile)
oa_age_bands_df = pd.read_csv(oa_age_bands_csv)

# Ensure population CSV column names are consistent
oa_age_bands_df.columns = ['OA21CD', 'Population']

# Merge the output area population data with the OA GeoDataFrame
print("Merging output area population data with OA GeoDataFrame...")
oa_gdf = oa_gdf.merge(oa_age_bands_df, on='OA21CD', how='left')

# Rename the columns to make sure we are clear
oa_gdf.rename(columns={'Population_x': 'Park_Population', 'Population_y': 'OA_Population'}, inplace=True)

# Print columns of oa_gdf to verify
print("Columns in oa_gdf:")
print(oa_gdf.columns)

# Print the first few rows of oa_gdf to inspect the data
print("First few rows of oa_gdf:")
print(oa_gdf.head())

# Calculate the population per address
print("Calculating population per address...")
oa_gdf['pop_per_address'] = oa_gdf['OA_Population'] / oa_gdf['total_addr']

# Calculate the population inside and outside the national parks
oa_gdf['pop_inside'] = oa_gdf['pop_per_address'] * oa_gdf['address_in']
oa_gdf['pop_outside'] = oa_gdf['pop_per_address'] * oa_gdf['address_ou']

# Print the first few rows of the updated oa_gdf to inspect the calculations
print("First few rows of the updated oa_gdf:")
print(oa_gdf[['OA21CD', 'OA_Population', 'total_addr', 'address_in', 'address_ou', 'pop_per_address', 'pop_inside', 'pop_outside']].head())

# Save the updated GeoDataFrame to a new shapefile
output_shapefile = './data/output_area_population_apportioned.shp'
print(f"Saving the updated GeoDataFrame to {output_shapefile}...")
oa_gdf.to_file(output_shapefile, driver='ESRI Shapefile')

Loading shapefile and CSVs...
Merging output area population data with OA GeoDataFrame...
Columns in oa_gdf:
Index(['OA21CD', 'LSOA21CD', 'LSOA21NM', 'LSOA21NMW', 'BNG_E', 'BNG_N', 'LAT',
       'LONG', 'GlobalID', 'index_righ', 'CODE', 'NAME', 'MEASURE',
       'DESIG_DATE', 'HOTLINK', 'STATUS', 'National_P', 'Park_Population',
       'address_in', 'total_addr', 'address_ou', 'geometry', 'OA_Population'],
      dtype='object')
First few rows of oa_gdf:
      OA21CD   LSOA21CD        LSOA21NM LSOA21NMW   BNG_E   BNG_N      LAT  \
0  E00027390  E01005409     Oldham 020D      None  402127  403557  53.5287   
1  E00027395  E01005410     Oldham 006A      None  401338  408961  53.5773   
2  E00027396  E01005410     Oldham 006A      None  402683  407583  53.5649   
3  E00027401  E01005410     Oldham 006A      None  401412  406686  53.5568   
4  E00029923  E01005908  Stockport 017D      None  398819  387648  53.3857   

      LONG                              GlobalID  index_righ  ...  DESIG_

C:\Users\alexad\AppData\Local\Temp\ipykernel_25760\2913384284.py:46: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  oa_gdf.to_file(output_shapefile, driver='ESRI Shapefile')
c:\Users\alexad\AppData\Local\anaconda3\envs\np-os\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Field DESIG_DATE create as date field, though DateTime requested.
  ogr_write(
c:\Users\alexad\AppData\Local\anaconda3\envs\np-os\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Park_Population' to 'Park_Popul'
  ogr_write(
c:\Users\alexad\AppData\Local\anaconda3\envs\np-os\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'OA_Population' to 'OA_Populat'
  ogr_write(
c:\Users\alexad\AppData\Local\anaconda3\envs\np-os\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'pop_per_address' to 'pop_per_ad'
  ogr_write(
c:\Users\alexad\AppData\Local\anac

### Optomize the threshold

In [38]:

# Load the necessary data
oa_population_shapefile = './data/output_area_population_apportioned.shp'

# Load shapefile
print("Loading shapefile data...")
oa_gdf = gpd.read_file(oa_population_shapefile)

# Retain only the specified columns
columns_to_keep = [
    'OA21CD', 'LSOA21CD', 'LSOA21NM', 'LSOA21NMW', 'BNG_E', 'BNG_N', 'LAT', 'LONG', 'GlobalID', 
    'index_righ', 'CODE', 'NAME', 'MEASURE', 'DESIG_DATE', 'HOTLINK', 'STATUS', 'National_P', 
    'Park_Popul', 'address_in', 'total_addr', 'address_ou', 'OA_Populat', 'pop_per_ad', 
    'pop_inside', 'pop_outsid'
]
oa_gdf = oa_gdf[columns_to_keep]

# Calculate percentage of addresses inside the park
oa_gdf['percentage_in'] = oa_gdf['address_in'] / oa_gdf['total_addr']

# Define the thresholds
thresholds = [i / 20.0 for i in range(0, 20)] 

# Add threshold columns
for threshold in thresholds:
    threshold_col = f'Threshold_{int(threshold * 100)}'
    oa_gdf[threshold_col] = oa_gdf.apply(
        lambda row: row['OA_Populat'] if row['percentage_in'] >= threshold else '', axis=1
    )

# Save the results to a CSV file
oa_gdf.to_csv('threshold_results.csv', index=False)

print("Threshold results saved to 'threshold_results.csv'.")

Loading shapefile data...
Threshold results saved to 'threshold_results.csv'.
